# Sea Surface Temperature from Remote Zarr Store

This example demonstrates using `PyVistaXarraySource` to lazily visualize a large, cloud-hosted sea surface temperature (SST) dataset stored as Zarr. The data is accessed on-demand — only the portion needed for rendering is loaded into memory.

Along the way, we explore several features of the source:

- **Resolution control** — downsample massive grids for fast previews
- **Time stepping** — move through temporal snapshots
- **Spatial slicing** — extract a region of interest with strided access
- **Mutable properties** — reconfigure the source after creation
- **Direct plotter integration** — pass the source to a `pv.Plotter` as a VTK algorithm
- **Accessor shorthand** — create a source via the `.pyvista.algorithm()` method

In [ ]:
import pyvista as pv
import xarray as xr

import pvxarray  # noqa: F401
from pvxarray.vtk_source import PyVistaXarraySource

## Load Remote Data

Open the NOAA CoastWatch Geo-Polar SST dataset from the Pangeo cloud data catalog. Using `chunks={}` enables dask-backed lazy loading so nothing is fetched until we ask for it:

In [ ]:
url = "https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/noaa-coastwatch-geopolar-sst-feedstock/noaa-coastwatch-geopolar-sst.zarr"
dataset = xr.open_dataset(url, engine="zarr", chunks={})
dataset

## Create the Source

`PyVistaXarraySource` wraps the DataArray as a VTK pipeline source. We map coordinate names to axes and set `resolution=0.01` so only ~1% of the data is sampled for a fast preview:

In [ ]:
source = PyVistaXarraySource(
    data_array=dataset["analysed_sst"],
    x="lat",
    y="lon",
    time="time",
    resolution=0.01,
)
source.time_index = 0

print(source)

## Render the SST Field

Call `.apply()` to execute the pipeline and produce a PyVista mesh. Because `resolution=0.01`, only a small fraction of the remote data is fetched:

In [ ]:
mesh = source.apply()
mesh.plot(cmap="coolwarm")

## Adjusting Resolution

All properties on `PyVistaXarraySource` are mutable. Changing any property automatically invalidates cached data so the next `.apply()` recomputes from scratch.

Increase the resolution to 5% to see more detail:

In [ ]:
source.resolution = 0.05

mesh = source.apply()
print(f"Mesh has {mesh.n_points:,} points at 5% resolution")
mesh.plot(cmap="coolwarm")

## Time Stepping

When a `time` dimension is specified, the source exposes `time_index` and `max_time_index` for navigating through temporal snapshots. Each time you change `time_index`, the cached data is cleared and the next `.apply()` fetches the new time step:

In [ ]:
print(f"Time dimension has {source.max_time_index + 1} steps (0 to {source.max_time_index})")

# Jump to a later time step
source.time_index = 100
source.resolution = 0.01  # keep it fast

mesh = source.apply()
mesh.plot(cmap="coolwarm")

## Spatial Slicing

The `slicing` property lets you extract a spatial region of interest using `{dim_name: [start, stop, step]}` index ranges. This is applied *before* the mesh is built, so only the requested region is loaded.

When slicing is set, it replaces the resolution-based downsampling — use the `step` value in each slice to control stride:

In [ ]:
source.time_index = 0
source.resolution = None  # disable resolution; slicing controls sampling
source.slicing = {
    "lat": [1000, 2500, 4],  # subset of latitude indices, every 4th point
    "lon": [2000, 5000, 4],  # subset of longitude indices, every 4th point
}

mesh = source.apply()
print(f"Sliced mesh: {mesh.n_points:,} points")
mesh.plot(cmap="coolwarm")

## Direct Plotter Integration

Because `PyVistaXarraySource` is a VTK algorithm, it can be passed directly to `pv.Plotter.add_mesh()`. PyVista treats it as a pipeline source and calls `Update()` during rendering. This means you can modify the source properties and re-render without manually calling `.apply()`:

In [ ]:
# Reset to simple resolution-based mode
source.slicing = None
source.resolution = 0.02
source.time_index = 0

pl = pv.Plotter()
pl.add_mesh(source, cmap="coolwarm")
pl.view_xy()
pl.show()

## Accessor Shorthand

The `.pyvista.algorithm()` method on any `xr.DataArray` is a convenient factory that creates a `PyVistaXarraySource` in one call. It accepts all the same parameters:

In [ ]:
source2 = dataset["analysed_sst"].pyvista.algorithm(
    x="lat",
    y="lon",
    time="time",
    resolution=0.01,
)
source2.time_index = 0

mesh = source2.apply()
mesh.plot(cmap="coolwarm")

## Inspecting Intermediate State

The source exposes its intermediate computation stages as read-only properties, which is useful for debugging or understanding what data will be visualized:

- `sliced_data_array` — the DataArray after time indexing, z-indexing, and spatial slicing/resolution downsampling
- `persisted_data` — the sliced data materialized into memory (for dask arrays, this calls `.persist()`)
- `data_range` — the `(min, max)` of the persisted values

In [ ]:
print("Sliced DataArray shape:", source2.sliced_data_array.shape)
print("Data range:", source2.data_range)